In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import Filtering
import Tool

In [22]:
def cnt(row, Rate_EST):
    if row["estimation"] == Rate_EST:
        return 1
    else:
        return 0

In [23]:
def ave_tem(data):
    cnt = 1
    total = 0
    for tem, k in zip(data["temperature"], data["estimation"]):
        if k > 2:
            cnt += 1
            total += tem
    return total / cnt

In [25]:
#################################################################################

In [26]:
withmax = 1

In [28]:
customer_iid = [1642, 4373, 6139, 7719, 8156]
for i in range(5):
    epochs = 3
    customer_id = customer_iid[i]
    Rate_EST = 3.0

    for epoch in range(epochs):

        data = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}.csv".format(
                customer_id
            )
        )

        acfilter = Filtering.ACfilter()
        helper = Tool.Disaggregation_tool()
        data = acfilter.ACfiltering(data)

        data_base = data[["Month", "consumption"]]
        base_load = data_base.groupby(["Month"], as_index=False).min()
        base_load.rename(columns={"consumption": "base_load"}, inplace=True)
        data = pd.merge(data, base_load, on=["Month"])

        dic = dict()
        for i in range(len(base_load)):
            dic[(base_load.iloc[i]["Month"])] = base_load.iloc[i]["base_load"]

        data["under"] = data.apply(
            lambda x: 1 if x["consumption"] < x["base_load"] + Rate_EST else 0, axis=1
        )
        data_NEV = data[data["under"] == 1]

        data_group = data_NEV.groupby(
            by=["Month", "Hour", "Minute"], as_index=False
        ).mean()

        data_group["consumption"] = data_group["consumption"]

        data_group = data_group[["Month", "Hour", "Minute", "consumption"]]
        data_total = pd.merge(data, data_group, on=["Month", "Hour", "Minute"])
        data_total = data_total.sort_values(by=["Month", "Day", "Hour", "Minute"])
        data_total["prev1"] = data_total["consumption_x"].shift(1)
        data_total["dif1"] = data_total["consumption_x"] - data_total["prev1"]
        data_total["prev2"] = data_total["consumption_x"].shift(2)
        data_total["dif2"] = data_total["consumption_x"] - data_total["prev2"]

        data_total.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
                customer_id
            ),
            index=False,
        )
        data = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
                customer_id
            )
        )

        ##################
        data = data.fillna(0)
        ##################

        data["higher"] = data.apply(
            lambda x: helper.charging_status(x, Rate_EST), axis=1
        )
        data["start"] = data.apply(
            lambda x: helper.change_point_status_start(x, Rate_EST), axis=1
        )
        data["end"] = data.apply(
            lambda x: helper.change_point_status_end(x, Rate_EST), axis=1
        )
        start_candidate = []
        candidate = []

        for i in range(len(data)):
            if data.iloc[i]["start"] == 1 and data.iloc[i]["higher"] == 1:
                start_candidate.append(i)

        visited = set()
        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1 and data.iloc[end]["higher"] <= 0
                ) and start not in start_visited:
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1
                    and data.iloc[end + 1]["end"] == -1
                    and data.iloc[end + 2]["end"] == -1
                    and start not in start_visited
                ):
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        candidate = helper.consecutive_filter(candidate)

        EVcharging = []
        for start, end in candidate:
            mean, std = helper.distribution(data, start, end, visited)
            if helper.ChargingDetection(data, start, end, mean, std, Rate_EST, dic):
                EVcharging.append([start, end - 1])

        Chargingrate = []
        cr1 = []
        for start, end in EVcharging:
            Chargingrate.append(max(data.iloc[start]["dif2"], data.iloc[start]["dif1"]))
            cr1.append(max(data.iloc[start]["dif2"], data.iloc[start]["dif1"]))

        lower = np.percentile(Chargingrate, 25)
        upper = np.percentile(Chargingrate, 75)
        Rate_EST = np.mean([i for i in Chargingrate if lower < i < upper])
        if withmax == 1:
            Rate_EST = max(Rate_EST, 3)

        #############################
        dif = []
        visited = set()
        for start, end in EVcharging:
            for k in range(start, end + 1):
                if k not in visited:
                    dif.append(
                        data.iloc[k]["consumption_x"] - data.iloc[k]["consumption_y"]
                    )
                    visited.add(k)

        Charging_period = set()
        bound = set()
        for start, end in EVcharging:
            for k in range(start, end + 1):
                Charging_period.add(k)
            bound.add(start - 1)
            bound.add(end + 1)

        data["estimation"] = helper.estimation(
            Rate_EST, len(data), Charging_period, bound
        )
        avg_temperature = ave_tem(data)

        Non_EVPeriods = data[data["estimation"] < 1]
        Non_EVPeriods = Non_EVPeriods[
            [
                "dataid",
                "local_15min",
                "grid",
                "solar",
                "car",
                "Month",
                "Day",
                "Hour",
                "Minute",
                "temperature",
                "weekday",
                "consumption_x",
            ]
        ]
        Non_EVPeriods.rename(columns={"consumption_x": "consumption"}, inplace=True)

        Non_EVPeriods.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Updated.csv".format(
                customer_id
            ),
            index=False,
        )
        data.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Estimation_Month.csv".format(
                customer_id
            ),
            index=False,
        )

        ######################UPDATE#####################

        if epoch == epochs - 1:
            break

        data = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Updated.csv".format(
                customer_id
            )
        )
        user = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}.csv".format(
                customer_id
            )
        )

        base_load = data.groupby(["Month"], as_index=False).min()[
            ["Month", "consumption"]
        ]
        base_load.rename(columns={"consumption": "base_load"}, inplace=True)
        user = pd.merge(user, base_load, how="left", on=["Month"])
        data_group = data.groupby(by=["Month", "Hour", "Minute"], as_index=False).mean()

        data_group = data_group[["Month", "Hour", "Minute", "consumption"]]
        data_group = pd.merge(
            user, data_group, how="left", on=["Month", "Hour", "Minute"]
        )
        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])
        data_group["prev1"] = data_group["consumption_x"].shift(1)
        data_group["dif1"] = data_group["consumption_x"] - data_group["prev1"]
        data_group["prev2"] = data_group["consumption_x"].shift(2)
        data_group["dif2"] = data_group["consumption_x"] - data_group["prev2"]
        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])

        for_back = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/forward_{0}.csv".format(
                customer_id
            )
        )
        for_back = for_back[["Month", "Day", "Hour", "Minute", "forward", "backward"]]
        data_group = pd.merge(
            data_group, for_back, on=["Month", "Day", "Hour", "Minute"]
        )

        data_group.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
                customer_id
            ),
            index=False,
        )

    data["cnt"] = data.apply(lambda x: cnt(x, Rate_EST), axis=1)

    print(
        f"id:{customer_id}, mean:{np.round(np.mean(dif),2)}, median:{np.round(np.median(dif),2)}, Average Charging Hours/Week:{np.round(len(dif)/4/52, 1)}, Average Temperature:{np.round(avg_temperature,2)}"
    )

id:1642, mean:3.59, median:3.45, Average Charging Hours/Week:7.0, Average Temperature:21.76
id:4373, mean:3.7, median:3.59, Average Charging Hours/Week:9.4, Average Temperature:21.85
id:6139, mean:3.4, median:3.17, Average Charging Hours/Week:10.8, Average Temperature:23.42
id:7719, mean:3.59, median:3.38, Average Charging Hours/Week:2.4, Average Temperature:27.45
id:8156, mean:3.74, median:3.44, Average Charging Hours/Week:6.9, Average Temperature:20.71
